In [1]:
from surprise import NMF, Dataset, Reader, SVD, SVDpp
from surprise.model_selection import cross_validate, GridSearchCV
import pandas as pd
from collections import defaultdict

In [2]:
def load_data(file_path):
    df = pd.read_csv(file_path, usecols=['userId', 'movieId', 'rating'])
    return df

In [3]:
def prepare_data_for_surprise(dataframe):
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(dataframe[['userId', 'movieId', 'rating']], reader)
    return data

In [4]:
traindf = load_data('./datasets/training_data.csv')

data = prepare_data_for_surprise(traindf)

In [5]:
param_grid = {
    'n_factors': [15],
    'n_epochs': [30],
    #'init_mean': [0, 0.05],  #SVD feature 
    #'init_std_dev': [0.05, 0.1, 0.15],  #SVD feature
    #'lr_all': [0.005, 0.01],  #SVD feature 
    #'reg_all': [0.02, 0.05],  #SVD feature 
    'reg_pu' : [0.12],  #NMF feature
    'reg_qi' : [0.12],  #NMF feature
    'reg_bu' : [0.04],  #NMF feature
    'reg_bi' : [0.02],  #NMF feature
    'lr_bu' : [0.01],  #NMF feature
    'lr_bi' : [0.01]  #NMF feature
}

#gs = GridSearchCV(SVD, param_grid, measures = ['rmse', 'mae'], cv = 5)
gs = GridSearchCV(NMF, param_grid, measures = ['rmse', 'mae'], cv = 5)
#gs = GridSearchCV(SVDpp, param_grid, measures = ['rmse', 'mae'], cv = 5)

gs.fit(data)

print(f"Best RMSE score achieved: {gs.best_score['rmse']}")

print(f"Best parameters: {gs.best_params['rmse']}")

Best RMSE score achieved: 0.8831535876905636
Best parameters: {'n_factors': 15, 'n_epochs': 30, 'reg_pu': 0.12, 'reg_qi': 0.12, 'reg_bu': 0.04, 'reg_bi': 0.02, 'lr_bu': 0.01, 'lr_bi': 0.01}


In [6]:
testdf = load_data('./datasets/testing_data.csv')
trainset = data.build_full_trainset()
testset = [tuple(row) for row in testdf.itertuples(index=False)]

In [7]:
#algo = SVD(**gs.best_params['rmse'])
algo = NMF(**gs.best_params['rmse'])
#algo = SVDpp(**gs.best_params['rmse'])

algo.fit(trainset)

predictions = algo.test(testset)

In [8]:
def calculate_accuracy(predictions, threshold=0.5):
    correct_predictions = sum(abs(pred.est - pred.r_ui) <= threshold for pred in predictions)
    accuracy = correct_predictions / len(predictions)
    return accuracy

In [9]:
threshold = 0.5
test_accuracy = calculate_accuracy(predictions, threshold)

print(f"Accuracy (within ±{threshold} of actual rating): {test_accuracy:.2%}")

Accuracy (within ±0.5 of actual rating): 46.05%


In [10]:
def create_predictions_dataframe(predictions):
    predictions_data = [{'uid': pred.uid, 'iid': pred.iid, 'og_rating': pred.r_ui, 'nmf_rating': pred.est} for pred in predictions]
    return pd.DataFrame(predictions_data)

test_pred_df = create_predictions_dataframe(predictions)

In [11]:
df_movies = pd.read_csv('./datasets/Movies.csv')

In [12]:
def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    org_ratings = defaultdict(list)
    
    for row in predictions.itertuples(index=False):
        uid, iid, og_rating, nmf_rating = row.uid, row.iid, row.og_rating, row.nmf_rating
        top_n[uid].append((iid, nmf_rating))
        org_ratings[uid].append((iid, og_rating))
    
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)  
        top_n[uid] = user_ratings[:n]
    
    return top_n, org_ratings

In [13]:
def show_user_recommended_movies_in_df(predictions, user_id, n=10):
    top_n, org_ratings = get_top_n(predictions, n)
    
    if user_id in top_n:
        recommended_movies_ids = [movie_id for movie_id, _ in top_n[user_id]]
    else:
        return pd.DataFrame()
    
    recommended_movies_df = df_movies.loc[df_movies['movieId'].isin(recommended_movies_ids)]
    
    return recommended_movies_df

In [14]:
for i in range(1,500):
    print("**{}**".format(i))
    print(show_user_recommended_movies_in_df(test_pred_df, i, 5))

**1**
      movieId                                   title  \
46         50              Usual Suspects, The (1995)   
257       296                     Pulp Fiction (1994)   
520       608                            Fargo (1996)   
863      1136  Monty Python and the Holy Grail (1975)   
2310     3062                 Longest Day, The (1962)   

                           genres  
46         Crime|Mystery|Thriller  
257   Comedy|Crime|Drama|Thriller  
520   Comedy|Crime|Drama|Thriller  
863      Adventure|Comedy|Fantasy  
2310             Action|Drama|War  
**2**
      movieId                                              title  \
6253    46970  Talladega Nights: The Ballad of Ricky Bobby (2...   
6315    48516                               Departed, The (2006)   
7436    80906                                  Inside Job (2010)   
7697    89774                                     Warrior (2011)   
8063    99114                            Django Unchained (2012)   

                    

      movieId                                   title  \
661       866                            Bound (1996)   
685       903                          Vertigo (1958)   
863      1136  Monty Python and the Holy Grail (1975)   
899      1197              Princess Bride, The (1987)   
2209     2936               Sullivan's Travels (1941)   

                                       genres  
661              Crime|Drama|Romance|Thriller  
685            Drama|Mystery|Romance|Thriller  
863                  Adventure|Comedy|Fantasy  
899   Action|Adventure|Comedy|Fantasy|Romance  
2209                 Adventure|Comedy|Romance  
**20**
      movieId                              title  \
792      1035         Sound of Music, The (1965)   
3010     4027  O Brother, Where Art Thou? (2000)   
3568     4886              Monsters, Inc. (2001)   
3640     4995           Beautiful Mind, A (2001)   
3705     5110              Super Troopers (2001)   

                                           genres

      movieId                                    title  \
97        110                        Braveheart (1995)   
2007     2671                      Notting Hill (1999)   
2302     3052                             Dogma (1999)   
3461     4720                       Others, The (2001)   
4480     6618  Shaolin Soccer (Siu lam juk kau) (2001)   

                             genres  
97                 Action|Drama|War  
2007                 Comedy|Romance  
2302       Adventure|Comedy|Fantasy  
3461  Drama|Horror|Mystery|Thriller  
4480                  Action|Comedy  
**35**
     movieId                       title                  genres
46        50  Usual Suspects, The (1995)  Crime|Mystery|Thriller
55        62   Mr. Holland's Opus (1995)                   Drama
201      235              Ed Wood (1994)            Comedy|Drama
202      236          French Kiss (1995)   Action|Comedy|Romance
509      592               Batman (1989)   Action|Crime|Thriller
**36**
      movieId      

      movieId                           title  \
157       186              Nine Months (1995)   
559       673                Space Jam (1996)   
1693     2278                    Ronin (1998)   
1794     2393  Star Trek: Insurrection (1998)   
1807     2407                   Cocoon (1985)   

                                                 genres  
157                                      Comedy|Romance  
559   Adventure|Animation|Children|Comedy|Fantasy|Sc...  
1693                              Action|Crime|Thriller  
1794                        Action|Drama|Romance|Sci-Fi  
1807                                      Comedy|Sci-Fi  
**56**
     movieId                     title                     genres
61        69             Friday (1995)                     Comedy
291      333          Tommy Boy (1995)                     Comedy
395      454          Firm, The (1993)             Drama|Thriller
430      493  Menace II Society (1993)         Action|Crime|Drama
592      733        

      movieId                                              title  \
210       246                                 Hoop Dreams (1994)   
267       307   Three Colors: Blue (Trois couleurs: Bleu) (1993)   
898      1196  Star Wars: Episode V - The Empire Strikes Back...   
921      1220                         Blues Brothers, The (1980)   
1431     1954                                       Rocky (1976)   

                       genres  
210               Documentary  
267                     Drama  
898   Action|Adventure|Sci-Fi  
921     Action|Comedy|Musical  
1431                    Drama  
**80**
      movieId                  title                       genres
6151    44191  V for Vendetta (2006)  Action|Sci-Fi|Thriller|IMAX
7043    69122   Hangover, The (2009)                 Comedy|Crime
7306    76251        Kick-Ass (2010)                Action|Comedy
7427    80831       Let Me In (2010)         Drama|Horror|Mystery
8218   103688  Conjuring, The (2013)              Horror|Thril

...

     movieId                               title  \
98       111                  Taxi Driver (1976)   
190      223                       Clerks (1994)   
796     1041               Secrets & Lies (1996)   
916     1215             Army of Darkness (1993)   
960     1261  Evil Dead II (Dead by Dawn) (1987)   

                                     genres  
98                     Crime|Drama|Thriller  
190                                  Comedy  
796                                   Drama  
916  Action|Adventure|Comedy|Fantasy|Horror  
960            Action|Comedy|Fantasy|Horror  
**373**
     movieId                       title                   genres
98       111          Taxi Driver (1976)     Crime|Drama|Thriller
123      150            Apollo 13 (1995)     Adventure|Drama|IMAX
242      280  Murder in the First (1995)           Drama|Thriller
334      377                Speed (1994)  Action|Romance|Thriller
398      457        Fugitive, The (1993)                 Thriller
**374**

      movieId                                              title  \
692       910                            Some Like It Hot (1959)   
908      1207                       To Kill a Mockingbird (1962)   
3984     5618  Spirited Away (Sen to Chihiro no kamikakushi) ...   
4909     7361       Eternal Sunshine of the Spotless Mind (2004)   
6315    48516                               Departed, The (2006)   

                           genres  
692                  Comedy|Crime  
908                         Drama  
3984  Adventure|Animation|Fantasy  
4909         Drama|Romance|Sci-Fi  
6315         Crime|Drama|Thriller  
**399**
      movieId                                          title  \
0           1                               Toy Story (1995)   
969      1270                      Back to the Future (1985)   
1939     2571                             Matrix, The (1999)   
4137     5952  Lord of the Rings: The Two Towers, The (2002)   
4360     6377                            Findin

      movieId                             title  \
123       150                  Apollo 13 (1995)   
257       296               Pulp Fiction (1994)   
277       318  Shawshank Redemption, The (1994)   
899      1197        Princess Bride, The (1987)   
3650     5014                   I Am Sam (2001)   

                                       genres  
123                      Adventure|Drama|IMAX  
257               Comedy|Crime|Drama|Thriller  
277                               Crime|Drama  
899   Action|Adventure|Comedy|Fantasy|Romance  
3650                                    Drama  
**427**
      movieId                             title  \
1158     1527         Fifth Element, The (1997)   
3624     4975                Vanilla Sky (2001)   
3676     5064  The Count of Monte Cristo (2002)   
3854     5418       Bourne Identity, The (2002)   
3890     5464          Road to Perdition (2002)   

                               genres  
1158   Action|Adventure|Comedy|Sci-Fi  
3624  Myst

     movieId                                      title  \
16        17               Sense and Sensibility (1995)   
31        32  Twelve Monkeys (a.k.a. 12 Monkeys) (1995)   
116      141                       Birdcage, The (1996)   
546      648                 Mission: Impossible (1996)   
577      708        Truth About Cats & Dogs, The (1996)   

                                genres  
16                       Drama|Romance  
31             Mystery|Sci-Fi|Thriller  
116                             Comedy  
546  Action|Adventure|Mystery|Thriller  
577                     Comedy|Romance  
**452**
      movieId                     title                            genres
474       541       Blade Runner (1982)            Action|Sci-Fi|Thriller
1067     1387               Jaws (1975)                     Action|Horror
1218     1617  L.A. Confidential (1997)  Crime|Film-Noir|Mystery|Thriller
1616     2160    Rosemary's Baby (1968)             Drama|Horror|Thriller
1883     2502       O

      movieId                               title  \
295       337  What's Eating Gilbert Grape (1993)   
621       786                       Eraser (1996)   
1398     1917                   Armageddon (1998)   
2355     3114                  Toy Story 2 (1999)   

                                           genres  
295                                         Drama  
621                         Action|Drama|Thriller  
1398               Action|Romance|Sci-Fi|Thriller  
2355  Adventure|Animation|Children|Comedy|Fantasy  
**479**
      movieId                       title                genres
650       838                 Emma (1996)  Comedy|Drama|Romance
686       904          Rear Window (1954)      Mystery|Thriller
931      1231     Right Stuff, The (1983)                 Drama
1503     2028  Saving Private Ryan (1998)      Action|Drama|War
1960     2599             Election (1999)                Comedy
**480**
      movieId                                   title  \
896      1193  On

In [15]:
show_user_recommended_movies_in_df(test_pred_df, 100, 5)

,movieId,title,genres
714,933,To Catch a Thief (1955),Crime|Mystery|Romance|Thriller
792,1035,"Sound of Music, The (1965)",Musical|Romance
899,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance
964,1265,Groundhog Day (1993),Comedy|Fantasy|Romance
4360,6377,Finding Nemo (2003),Adventure|Animation|Children|Comedy
